In [1]:
#Import Library

#Preprocessing
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import scipy.ndimage as ndi
from random import shuffle
from scipy.misc import imread, imresize
from scipy.io import loadmat

#Model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2

#Save Model
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [15]:
#Dataset Parameter
dataset_name = "imdb"
images_path = '.../imdb/imdb_crop/'

#Model parameters
batch_size = 32
num_epochs = 10
input_shape = (48, 48, 3)
validation_split = .2
verbose = 1
num_classes = 2
patience = 50

#path to save the models
base_path = '.../pretrained_models/'                         

# 1) Preprocessing (Define the Dataset/Train,Validation Split)

In [16]:
class DataManager(object):
    """Class for loading imdb gender classification dataset."""
    def __init__(self, dataset_name='imdb',
                 dataset_path=None, image_size=(48, 48)):

        self.dataset_name = dataset_name
        self.dataset_path = dataset_path
        self.image_size = image_size
        if self.dataset_path is not None:
            self.dataset_path = dataset_path
        elif self.dataset_name == 'imdb':
            self.dataset_path = '.../imdb/imdb_crop/imdb.mat'
        else:
            raise Exception(
                    'Incorrect dataset name, please input imdb dataset')

    def get_data(self):
        if self.dataset_name == 'imdb':
            ground_truth_data = self._load_imdb()
        return ground_truth_data

    def _load_imdb(self):
        face_score_treshold = 3
        dataset = loadmat(self.dataset_path)
        image_names_array = dataset['imdb']['full_path'][0, 0][0]
        gender_classes = dataset['imdb']['gender'][0, 0][0]
        face_score = dataset['imdb']['face_score'][0, 0][0]
        second_face_score = dataset['imdb']['second_face_score'][0, 0][0]
        face_score_mask = face_score > face_score_treshold
        second_face_score_mask = np.isnan(second_face_score)
        unknown_gender_mask = np.logical_not(np.isnan(gender_classes))
        mask = np.logical_and(face_score_mask, second_face_score_mask)
        mask = np.logical_and(mask, unknown_gender_mask)
        image_names_array = image_names_array[mask]
        gender_classes = gender_classes[mask].tolist()
        image_names = []
        for image_name_arg in range(image_names_array.shape[0]):
            image_name = image_names_array[image_name_arg][0]
            image_names.append(image_name)
        return dict(zip(image_names, gender_classes))

In [17]:
#Preprocessing functions to read the imdb facedataset

def get_labels(dataset_name):
    if dataset_name == 'imdb':
        return {0: 'woman', 1: 'man'}
    else:
        raise Exception('Invalid dataset name')


def get_class_to_arg(dataset_name='imdb'):
    if dataset_name == 'imdb':
        return {'woman': 0, 'man': 1}
    else:
        raise Exception('Invalid dataset name')

def split_imdb_data(ground_truth_data, validation_split=.2, do_shuffle=False):
    ground_truth_keys = sorted(ground_truth_data.keys())
    if do_shuffle is not False:
        shuffle(ground_truth_keys)
    training_split = 1 - validation_split
    num_train = int(training_split * len(ground_truth_keys))
    train_keys = ground_truth_keys[:num_train]
    validation_keys = ground_truth_keys[num_train:]
    return train_keys, validation_keys
   

In [5]:
# Initialize object
data_loader = DataManager(dataset_name)
ground_truth_data = data_loader.get_data()           #get_data (image_name_path , gender)
ground_truth_data

{'02/nm0000002_rm1346607872_1924-9-16_2004.jpg': 0.0,
 '02/nm0000002_rm1363385088_1924-9-16_2004.jpg': 0.0,
 '02/nm0000002_rm221957120_1924-9-16_1974.jpg': 0.0,
 '02/nm0000002_rm2287049216_1924-9-16_2007.jpg': 0.0,
 '02/nm0000002_rm238734336_1924-9-16_1974.jpg': 0.0,
 '02/nm0000002_rm2585828096_1924-9-16_2006.jpg': 0.0,
 '02/nm0000002_rm2769394176_1924-9-16_2006.jpg': 0.0,
 '02/nm0000002_rm2780403712_1924-9-16_2004.jpg': 0.0,
 '02/nm0000002_rm2983566080_1924-9-16_2007.jpg': 0.0,
 '02/nm0000002_rm4111175424_1924-9-16_2010.jpg': 0.0,
 '02/nm0000002_rm4213815808_1924-9-16_1990.jpg': 0.0,
 '02/nm0000002_rm505206272_1924-9-16_1991.jpg': 0.0,
 '02/nm0000002_rm800626688_1924-9-16_2006.jpg': 0.0,
 '02/nm0000002_rm81958912_1924-9-16_2005.jpg': 0.0,
 '02/nm0000002_rm991745536_1924-9-16_1991.jpg': 0.0,
 '03/nm0000003_rm1883436032_1934-9-28_1963.jpg': 0.0,
 '03/nm0000003_rm2034430976_1934-9-28_1963.jpg': 0.0,
 '03/nm0000003_rm3428825856_1934-9-28_1965.jpg': 0.0,
 '04/nm0000004_rm1570360832_1949-1-

In [6]:
train_keys, val_keys = split_imdb_data(ground_truth_data, validation_split)

train_keys
val_keys

print('Number of training samples:', len(train_keys))
print('Number of validation samples:', len(val_keys))

Number of training samples: 74820
Number of validation samples: 18706


# 2) Preprocessing (Encode the Dataset)

In [7]:
x_train = []
y_train = []


for key in train_keys:
    image_path =  images_path + key
    image_array = cv2.imread(image_path)
    image_array = imresize(image_array, (48, 48))
    num_image_channels = len(image_array.shape)
    
    #image_array = cv2.cvtColor(image_array,cv2.COLOR_RGB2GRAY).astype('float32')

    ground_truth = ground_truth_data[key]
    image_array = image_array.astype('float32')

    x_train.append(image_array)    
    y_train.append(ground_truth)
               

C:\Users\DELL\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


In [8]:
#validation

x_valid = []
y_valid = []

for key in val_keys:
    image_path =  images_path + key
    image_array = cv2.imread(image_path)
    image_array = imresize(image_array, (48, 48))
    num_image_channels = len(image_array.shape)
    
    #image_array = cv2.cvtColor(image_array,cv2.COLOR_RGB2GRAY).astype('float32')
    
    #image_array = np.expand_dims(image_array, -1)
    ground_truth = ground_truth_data[key]
    image_array = image_array.astype('float32')
    
    x_valid.append(image_array)
    y_valid.append(ground_truth)
               

C:\Users\DELL\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if __name__ == '__main__':


In [9]:
#Preprocess Train & Valid Data

#Normalize the input features (x_train,x_valid)
xtrain = np.array(x_train) / 255
xvalid = np.array(x_valid) / 255

#Reshape the input features(x_train)
w, h = 48, 48
x_train = xtrain.reshape(xtrain.shape[0], w, h, 3)
x_valid = xvalid.reshape(xvalid.shape[0], w, h, 3)
print("Shape of x_train: ", x_train.shape)
print("Shape of x_valid: ", x_valid.shape)

#Categorical encode the input label(y_train)
ytrain = np.array(y_train)
yvalid = np.array(y_valid)

y_train = tf.keras.utils.to_categorical(ytrain, 2)
y_valid = tf.keras.utils.to_categorical(yvalid, 2)
print("Shape of y_train: ", y_train.shape)
print("Shape of y_valid: ", y_valid.shape)

Shape of x_train:  (74820, 48, 48, 3)
Shape of x_valid:  (18706, 48, 48, 3)
Shape of y_train:  (74820, 2)
Shape of y_valid:  (18706, 2)


# 3) Train the Model

In [18]:
#Data Augmentation
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

In [19]:
def cnn(input_shape, num_classes, l2_regularization=0.01):
    regularization = l2(l2_regularization)

    # base
    img_input = Input(input_shape)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
                                            use_bias=False)(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
                                            use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # module 1
    residual = Conv2D(16, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 2
    residual = Conv2D(32, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 3
    residual = Conv2D(64, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 4
    residual = Conv2D(128, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    x = Conv2D(num_classes, (3, 3),
            #kernel_regularizer=regularization,
            padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output = Activation('softmax',name='predictions')(x)

    model = Model(img_input, output)
    return model


In [20]:
# model parameters/compilation
model = cnn(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 48, 48, 3)    0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 46, 46, 8)    216         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 46, 46, 8)    32          conv2d_8[0][0]                   
__________________________________________________________________________________________________
activation_7 (Activation)       (None, 46, 46, 8)    0           batch_normalization_15[0][0]     
__________________________________________________________________________________________________
conv2d_9 (

In [21]:
# callbacks
log_file_path = base_path + '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
trained_models_path = base_path + '_cnn'
model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

In [23]:
num_epochs = 20

#Train the model
model.fit_generator(data_generator.flow(x_train, y_train,
                                            batch_size),
                        steps_per_epoch=len(x_train) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=(x_valid,y_valid))


Epoch 1/20
2339/2338 [==============================] - 82s 35ms/step - loss: 0.1795 - acc: 0.9401 - val_loss: 0.1739 - val_acc: 0.9411

Epoch 00001: val_loss did not improve from 0.16863
Epoch 2/20
2339/2338 [==============================] - 82s 35ms/step - loss: 0.1760 - acc: 0.9414 - val_loss: 0.1685 - val_acc: 0.9446

Epoch 00002: val_loss improved from 0.16863 to 0.16850, saving model to E:/Emotion+Gender Classification/face_classification-master/gender_classifi_git/pretrained_models_cnn.02-0.94.hdf5
Epoch 3/20
2339/2338 [==============================] - 82s 35ms/step - loss: 0.1749 - acc: 0.9418 - val_loss: 0.1751 - val_acc: 0.9421

Epoch 00003: val_loss did not improve from 0.16850
Epoch 4/20
2339/2338 [==============================] - 82s 35ms/step - loss: 0.1728 - acc: 0.9419 - val_loss: 0.2031 - val_acc: 0.9291

Epoch 00004: val_loss did not improve from 0.16850
Epoch 5/20
2339/2338 [==============================] - 82s 35ms/step - loss: 0.1701 - acc: 0.9440 - val_loss: 0